In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True).cuda()

for param in model.parameters():
    param.requires_grad = False   

model.fc = nn.Sequential(
            nn.Linear(2048, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 42)
        ).to(device)

C:\Users\tanas\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in C:\Users\tanas/.cache\torch\hub\pytorch_vision_v0.10.0


In [2]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.fc.parameters())

In [3]:
from dataloader.CMUpanoptic import CMUDataset
dataset = CMUDataset("E:/Dataset/CMU/hand_syn/synth1", 64)

dataloader = DataLoader(dataset, 
                        batch_size = 1)

In [4]:
dat = next(iter(dataloader))
print(dat[0].shape)
print(dat[0].dtype)
print(dat[1].shape)

print(device)

torch.Size([1, 3, 64, 64])
torch.uint8
torch.Size([1, 42])
cuda


In [5]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device).float()
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataset)
        epoch_acc = running_corrects.double() / len(dataset)

        print('loss: {:.4f}, acc: {:.4f}'.format(epoch_loss,
                                                    epoch_acc))
    return model

In [7]:
model_trained = train_model(model, criterion, optimizer, num_epochs = 10)

Epoch 1/10
----------


100%|██████████| 6540/6540 [01:24<00:00, 77.65it/s]


loss: 39.7950, acc: 0.0046
Epoch 2/10
----------


100%|██████████| 6540/6540 [01:22<00:00, 78.81it/s]


loss: 39.5108, acc: 0.0000
Epoch 3/10
----------


...

loss: 38.2261, acc: 0.0000
Epoch 9/10
----------


100%|██████████| 6540/6540 [01:23<00:00, 78.77it/s]


loss: 38.3512, acc: 0.0000
Epoch 10/10
----------


100%|██████████| 6540/6540 [01:22<00:00, 78.92it/s]

loss: 38.0950, acc: 0.0000


In [9]:
checkpoint = { "state_dict": model.state_dict() }
torch.save(checkpoint, 'regressor/checkpoint.chk')